### 如何开发单词级神经语言模型并使用它来生成文本

一个语言模型可以预测序列中的下一个单词的概率。

神经网络模型是用于开发统计语言模型的首选方法，因为它们可以使用分布式表示，其中具有相似含义的不同单词具有相似的表示，并且它们在进行预测时可以使用最近观察到的单词的上下文。

#### 柏拉图的共和国
共和国是希腊古典哲学家柏拉图最著名的作品。

它的结构是一个对话（例如对话），涉及城市国家内的秩序与正义。

整个文本可在公共领域免费获得。可在Gutenberg项目网站上以多种格式获得。

文本应以如下内容开头：

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
…

以如下内容结尾：

…
And it shall be well with us both in this life and in the pilgrimage of a thousand years which we have been describing.

将清理后的版本另存为“ republic_clean”。txt”在您当前的工作目录中。该文件应为约15802行文本。

现在，我们可以根据该文本开发语言模型。

### 语言模型设计
在本教程中，我们将开发一个文本模型，然后将其用于生成新的文本序列。

语言模型将是统计性的，并将在给定文本输入序列的情况下预测每个单词的概率。预测的单词将作为输入输入，进而生成下一个单词。

一个关键的设计决定是输入序列应该有多长。它们需要足够长，以使模型能够学习用于预测单词的上下文。当我们使用模型时，此输入长度还将定义用于生成新序列的种子文本的长度。

没有正确的答案。如果有足够的时间和资源，我们可以探索模型学习不同大小的输入序列的能力。

取而代之的是，我们将随意选择50个单词的长度作为输入序列的长度。

我们可以处理数据，以便该模型只处理独立的句子，并填充或截断文本以满足每个输入序列的这一要求。您可以将其作为本教程的扩展进行探索。

相反，为了使示例简短，我们将让所有文本一起流动并训练模型以预测文本中句子，段落甚至书籍或章节中的下一个单词。

现在我们有了模型设计，我们可以看一下如何将原始文本转换为由50个输入单词到1个输出单词的序列，准备适合模型。

### 载入文字
第一步是将文本加载到内存中。

我们可以开发一个小函数来将整个文本文件加载到内存中并返回它。该函数称为load_doc（），并在下面列出。给定文件名，它将返回一系列已加载的文本。

In [1]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [2]:
# load document
in_filename = './input/republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


### 预处理

我们需要将原始文本转换为标记或单词序列，以用作训练模型的源。

在查看原始文本的基础上，以下是我们将执行的一些清理文本的特定操作。作为扩展，您可能想自己探索更多操作。

- 用空格替换“ –”，以便我们更好地拆分单词。
- 根据空白拆分单词。
- 删除单词中的所有标点符号以减小词汇量（例如，“ What？”变为“ What”）。
- 删除所有非字母的单词，以删除独立的标点符号。
- 将所有单词标准化为小写以减小词汇量。

词汇量对语言建模来说很重要。词汇量越小，模型的训练速度就越快。

我们可以按一个函数的顺序执行这些清洁操作。下面是函数clean_doc（）。

In [3]:
import string
 
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [4]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

### 保存纯文本
我们可以将长长的令牌列表组织成50个输入字和1个输出字的序列。

即51个单词的序列。

为此，我们可以迭代从令牌51开始的令牌列表，并将之前的50个令牌作为序列，然后重复此过程直到令牌列表的末尾。

我们会将令牌转换成以空格分隔的字符串，以供以后存储在文件中。

下面列出了将干净令牌列表分成长度为51个令牌的序列的代码。

In [5]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 118633


In [6]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

In [7]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

使用您的文本编辑器查看文件。

您会看到每一行都沿一个单词移动，最后有一个新单词可以预测

### 语言模型
现在，我们可以从准备的数据中训练统计语言模型。

我们将训练的模型是神经语言模型。它具有一些独特的特征：

它使用单词的分布式表示，以便具有相似含义的不同单词将具有相似的表示。
它在学习模型的同时学习表示。
它学习使用最近100个单词的上下文来预测下一个单词的概率。
具体来说，我们将使用嵌入层来学习单词的表示，并使用长短期记忆（LSTM）递归神经网络来学习根据上下文来预测单词。

让我们从加载训练数据开始。

#### 加载顺序
我们可以使用上一节中开发的load_doc（）函数加载训练数据。

加载后，我们可以根据新行进行拆分，将数据拆分为单独的训练序列。

下面的代码片段将从当前工作目录中加载“ republic_sequences.txt ”数据文件。

In [8]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [10]:
lines[0]

'book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was'

#### 编码序列
单词嵌入层期望输入序列由整数组成。

我们可以将词汇表中的每个单词映射到唯一的整数并编码输入序列。稍后，当我们进行预测时，我们可以将预测转换为数字，并在同一映射中查找它们的关联单词。

为了进行这种编码，我们将使用Keras API中的Tokenizer类。

首先，必须在整个训练数据集上训练Tokenizer，这意味着它会找到数据中所有唯一的单词，并为每个单词分配一个唯一的整数。

然后，我们可以使用fit Tokenizer对所有训练序列进行编码，将每个序列从单词列表转换为整数列表。

In [9]:
from keras.preprocessing.text import Tokenizer

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

/Users/likaiwei/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
sequences[0]

[1046,
 11,
 11,
 1045,
 329,
 7409,
 4,
 1,
 2873,
 35,
 213,
 1,
 261,
 3,
 2251,
 9,
 11,
 179,
 817,
 123,
 92,
 2872,
 4,
 1,
 2249,
 7408,
 1,
 7407,
 7406,
 2,
 75,
 120,
 11,
 1266,
 4,
 110,
 6,
 30,
 168,
 16,
 49,
 7405,
 1,
 1609,
 13,
 57,
 8,
 549,
 151,
 11,
 57]

我们可以将单词映射为整数，作为Tokenizer对象上名为word_index的字典属性。

我们需要知道词汇表的大小，以便以后定义嵌入层。我们可以通过计算映射字典的大小来确定词汇量。

为单词分配的值从1到单词总数（例如7,409）。嵌入层需要为此词汇表中的每个单词从索引1到最大索引分配一个矢量表示，并且由于数组的索引是零偏移量，因此单词末尾的单词索引将为7,409；这意味着数组的长度必须为7,409 + 1。

因此，当为“嵌入”层指定词汇量时，我们将其指定为比实际词汇量大1。

In [17]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

### 序列输入和输出

现在我们已经对输入序列进行了编码，我们需要将它们分为输入（X）和输出（y）元素。

我们可以通过数组切片来做到这一点。

分离后，我们需要对输出字进行热编码。这意味着将其从整数转换为向量，词汇表中每个单词对应一个值，其中1表示在单词整数值的索引处的特定单词。

这样一来，模型就可以预测下一个单词的概率分布。

Keras提供了to_categorical（），可用于对每个输入-输出序列对的输出字进行热编码。

最后，我们需要向Embedding层指定输入序列的长度。我们知道有50个词是因为我们设计了模型，但是一种好的通用方法是使用输入数据形状的第二维（列数）。这样，如果在准备数据时更改了序列的长度，则无需更改此数据加载代码。它是通用的。

In [19]:
from numpy import array
from keras.utils import to_categorical

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

### 拟合模型
现在，我们可以在训练数据上定义和拟合我们的语言模型。

学习的嵌入需要知道词汇量和输入序列的长度，如前所述。它还具有一个参数，用于指定将使用多少维来表示每个单词。即，嵌入向量空间的大小。

常见值为50、100和300。在这里我们将使用50，但考虑测试较小或较大的值。

我们将使用两个LSTM隐藏层，每个包含100个存储单元。更多的存储单元和更深的网络可能会获得更好的结果。

具有100个神经元的密集的完全连接层连接到LSTM隐藏层，以解释从序列中提取的特征。输出层将下一个单词作为词汇量大小的单个矢量预测，并为词汇中的每个单词提供概率。softmax激活函数用于确保输出具有归一化概率的特征。

In [21]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            370500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
118633/118633 [==============================] - 597s 5ms/step - loss: 5.7583 - acc: 0.1037
Epoch 2/10
118633/118633 [==============================] - 552s 5ms/step - loss: 5.4834 - acc: 0.1331
Epoch 3/10
118633/118633 [==============================] - 480s 4ms/step - loss: 5.2847 - acc: 0.1480
Epoch 4/10
118633/118633 [==============================] - 494s 4ms/step - loss: 5.1461 - acc: 0.1594
Epoch 5/10
118633/118633 [==============================] - 489s 4ms/step - loss: 5.0340 - acc: 0.1667
Epoch 6/10
118633/118633 [==============================] - 487s 4ms/step - loss: 4.9369 - acc: 0.1727
Epoch 7/10
118633/118633 [==============================] - 7101s 60ms/step - loss: 4.8462 - acc: 0.1780
Epoch 8/10
118633/118633 [==============================] - 487s 4ms/step - loss: 4.7584 - acc: 0.1832
Epoch 9/10
118633/118633 [==============================] - 481s 4ms/step - loss: 4.6733 - acc: 0.1877
Epoch 10/10
118633/118633 [==============================] - 528s 4ms/s

### 保存模型
运行结束时，经过训练的模型将保存到file中。

在这里，我们使用Keras模型API将模型保存到当前工作目录中的文件“ model.h5 ”。

稍后，当我们加载模型进行预测时，我们还将需要将单词映射为整数。这在Tokenizer对象中，我们也可以使用Pickle保存它。

In [27]:
from pickle import dump

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

### 完整的例子
我们可以将所有这些放在一起。下面列出了适合语言模型的完整示例

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            370500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
118633/118633 [==============================] - 605s 5ms/step - loss: 6.1962 - acc: 0.0667
Epoch 2/100
118633/1186

### 使用语言模型
现在我们有了训练有素的语言模型，就可以使用它了。

在这种情况下，我们可以使用它来生成与源文本具有相同统计属性的新文本序列。

我们将从重新加载训练序列开始。

#### 载入资料
我们可以使用上一部分中的相同代码来加载文本的训练数据序列。

具体来说，就是load_doc（）函数。

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

我们需要文本，以便我们可以选择源序列作为模型输入，以生成新的文本序列。

该模型将需要100个单词作为输入。

稍后，我们将需要指定预期的输入长度。我们可以从输入序列中确定这一点，方法是计算已加载数据的一行的长度，并对同样在同一行的预期输出字减去1。

In [ ]:
seq_length = len(lines[0].split()) - 1

### 加载模型
现在，我们可以从文件中加载模型。

Keras提供了load_model（）函数来加载模型，以供使用。

In [ ]:
# load the model
model = load_model('model.h5')

In [ ]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

### 产生文字
生成文本的第一步是准备种子输入。

为此，我们将从输入文本中选择随机文本行。选择之后，我们将打印它，以便我们对使用的内容有所了解。

In [ ]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

接下来，我们可以一次生成一个新单词。

首先，必须使用与训练模型时相同的标记器将种子文本编码为整数。

In [ ]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]

该模型可以通过调用model.predict_classes（）直接预测下一个单词，该函数将以最高的概率返回该单词的索引。



In [ ]:
# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=0)

然后，我们可以在Tokenizers映射中查找索引以获取关联的单词。

In [ ]:
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break

然后，我们可以将此单词附加到种子文本中并重复该过程。

重要的是，输入序列将变得太长。在将输入序列编码为整数之后，我们可以将其截断为所需的长度。Keras提供了pad_sequences（）函数，可用于执行此截断。

In [ ]:
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')


我们可以将所有这些包装到一个称为generate_seq（）的函数中，该函数将模型，标记器，输入序列长度，种子文本和要生成的单词数作为输入。然后，它返回由模型生成的单词序列。



In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

放在一起，下面列出了用于从学习语言模型生成文本的完整代码清单。

In [28]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)
 
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
 
# load the model
model = load_model('model.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')
 
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

better be referred to damon himself for the analysis of the subject would be difficult you know socrates expresses himself carelessly in accordance with his assumed ignorance of the details of the subject in the first part of the sentence he appears to be speaking of paeonic rhythms which are in

the state and the same and the same and the same and the other and the other and the other and the other and the other and the other and the other and the other and the other and the other and the other and the other and the other


### 扩展
本节列出了一些扩展的想法。

- 句。根据句子拆分原始数据，并将每个句子填充到固定长度（例如最长的句子长度）。
- 简化词汇。探索更简单的词汇，去除词干或停用词。
- 音调模型。调整模型，例如嵌入的大小或隐藏层中的存储单元数，以查看是否可以开发更好的模型。
- 更深层次的模型。将模型扩展为具有多个LSTM隐藏层，也许可以通过Dropout来查看是否可以开发更好的模型。
- 预训练词嵌入。扩展模型以使用预训练的word2vec或GloVe向量，以查看是否可以得到更好的模型。